# Fusera variant calling pipeline
---

In this tutorial, we will use [Fusera](https://github.com/mitre/fusera) (a cloud extension to the [NCBI Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra)) to access publicly available test data from the 1000 Genomes project. Then we will follow GATK best practices using a tool called [bcbio](https://bcbio-nextgen.readthedocs.io/en/latest/index.html) to do variant calling on the test data.

**Note:** If you have not installed [Fusera](https://github.com/mitre/fusera) and [bcbio](https://bcbio-nextgen.readthedocs.io/en/latest/index.html), please do so before proceeding.

**Fusera is one option for accessing sequencing data, but it is NOT required to do variant calling with [bcbio](https://bcbio-nextgen.readthedocs.io/en/latest/index.html).**

---

## Part I: Accessing data with Fusera

First, let's set up directories for this tutorial. The directory `cloud_mountpoint` must be empty; we will use this directory to mount our cloud instance for data access. The `local_results` directory will store our output files from running GATK variant calling pipeline.

In [ ]:
# This is dir we will use to mount cloud instance to access data
mkdir -p ~/cloud_mountpoint # This dir must be empty for Fusera to work!

# We will use this local dir on to store our analysis output
mkdir -p ~/local_results

In [ ]:
ls

### Example: Open Access SRA Data

Now, we will mount the cloud instance to access our test data. **Important:** You must unmount the cloud instance after you are done. You can use the command `fusera unmount ~/cloud_mountpoint`. I have also included a code block at the end of this tutorial to unmount.

If you would like to access other data, you can find valid input for the `--location` flag by going to [NCBI SRA](https://www.ncbi.nlm.nih.gov/sra), then SRA Run Selector. Now you can search for your run (i.e. SRR1219902) and look at the RunInfo Table. You can use info in the `DATASTORE_region` column of the RunInfo Table for the `--location` flag. For the example used in this tutorial, the RunInfo Table can be found [on this page](https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRR121990&go=go).

For this example, we will be working with open access SRA data with run number: **SRR1597834**. For more info about this accession, please follow [this link](https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRR1597834&go=go).

In [ ]:
# You will have to change the filepaths depending on your directory structure
fusera mount --accession "SRR1597834" --location s3.us-east-1 ~/cloud_mountpoint > output.log 2>&1 &


#### Slight Tangent

In the example above, we are using open access data. If you are accessing data that requires authorized access, you will need to obtain a `.ngc` file. Go [here for more info](https://www.ncbi.nlm.nih.gov/books/NBK63512/) on `.ngc` files and [here to request for authorized access](https://dbgap.ncbi.nlm.nih.gov/aa/wga.cgi?page=login) from [dbGaP](https://www.ncbi.nlm.nih.gov/gap).

If you are working with authorized access data, you will need to modify the command line above like so:

```bash
fusera mount --ngc "~/prj_phs710EA_test.ngc" --accession "SRR1219902" --location s3.us-east-1 ~/cloud_mountpoint > output.log 2>&1 &
```

The test `.ngc` file can be found on [this page](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc&f=dbgap_use) and is associated with accession [SRR1219902](https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRR1219902&go=go). To download the `.ngc` file, you can use the following command:

```bash
# Download test .ngc file (publicly available)
cd ~
wget ftp://ftp.ncbi.nlm.nih.gov/sra/examples/decrypt_examples/prj_phs710EA_test.ngc
```

Now, getting back to our open access example with accession **SRR1597834**. Let's take a look at what is inside of `cloud_mountpoint`.

In [ ]:
cd ~/cloud_mountpoint
ls

---

## Part II: GATK variant calling

First, let's edit the `config_GCF_gatk_variant.yaml` (this file lives in the same directory as this iPython notebook). We will need to change lines 5 and 6.

**Step 1:** Change line 5 `dir: /home/ubuntu/local_results` to match your directory structure. For example, if the full filepath to your `local_results` directory is `/home/users/pacman/local_results`, your line 5 should look like: `dir: /home/users/pacman/local_results`.

You can find the full filepath of a directory by:

```bash
cd ~/local_results
pwd -P # This will output the full filepath to the directory you are currently in
```

**Step 2:** Change line 6 `files: [/home/ubuntu/cloud_mountpoint/SRR1597834/NA06985.mapped.ILLUMINA.bwa.CEU.low_coverage.20120522.bam]` to match your directory structure like we did above.

**Step 3:** Now, save the file and we are ready to run `bcbio`.

In [ ]:
# The -n flag distributes this across 8 local cores
# Here, we assume you are in the directory that contains config_GCF_gatk_variant.yaml
bcbio_nextgen.py config_GCF_gatk_variant.yaml -n 8

#### More on `bcbio`

If you would like to use a different variant calling pipeline, `bcbio` comes with many configuration file templates. Or you can also use the following command to automatically create a sample configuration file for your favorite pipeline:

```bash
bcbio_nextgen.py -w template gatk-variant project1 sample1.bam sample2_1.fq sample2_2.fq
```

You can find a list of supported pipelines [here](https://bcbio-nextgen.readthedocs.io/en/latest/contents/pipelines.html).

Please read [`bcbio`'s getting started page](https://bcbio-nextgen.readthedocs.io/en/latest/contents/testing.html) for more details.

---

## Important: Unmount Fusera

In [ ]:
disown %1

In [ ]:
cd ~/jupyter_notebooks/fusera_vc
fusera unmount ~/cloud_mountpoint